In [1]:
import json


In [2]:
cluster = [
    [7432321740.0],
    [6167522109.0],
    [5551467928.0],
    [2514969442.0],
    [1042580010.0, 931749218.0, 915045194.0, 910360336.0],
    [484014708.0, 312106827.0, 304406965.0, 260477815.0, 228769401.0, 196037656.0, 188246330.0, 182687057.0, 124618387.0, 105071824.0, 58522017.0, 58378179.0, 57389046.0, 46110565.0, 44702830.0, 41850608.0, 38570198.0, 29167362.0, 16747909.0, 13742728.0, 10806845.0, 10650500.0, 10359644.0, 9069314.0, 7251826.0, 6100002.0, 3111139.0, 3076673.0, 3062904.0, 2702001.0, 2511547.0, 2370896.0, 728920.0,
     583481.0, 419615.0, 419593.0, 348663, 301161.0, 272260.0, 239118.0, 205957.0, 198748.0, 96097.0, 95542.0, 90400.0, 88669.0, 79206.0, 69438.0, 54103.0, 43126.0, 37155.0, 29754.0, 28766.0, 27982.0, 26389.0, 25020, 15922.0, 14991.0, 13373.0, 11000.0, 8873.0, 7428.0, 3010.0, 2789.0, 2140.0, 1884.0, 1200.0, 1051.0, 1001.0, 928.0, 757.0, 729, 588.0, 558, 496.0, 425.0, 370.0, 350.0, 320.0, 181.0, 120.0, 90.0, 56.0, 38.0, 34.0, 33.0, 20.0, 17.0, 15.0, 11.0, 10.0, 6.0, 5.0, 4.0, 3, 3.0, 2.0, 1.0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
]


In [3]:
len([j for i in cluster for j in i])


124

In [4]:
class Foo:
    def __init__(self):
        self.nodes = [i for i in range(0, 124)]
        with open('src/data/network2/2011-attributes.json') as f:
            self.attributes = json.load(f)

    def set_attributes(self, attributes=None):

        # ... some code

        if attributes is None:
            self.attributes = {
                "IS": {"layer": 6, "cluster": cluster},
            }

    def set_attribute_probability(self):
        for key in self.attributes.keys():
            print(key)
            self.attributes[key]['p'] = [len(item) / len(self.nodes)
                                         for item in self.attributes[key]["cluster"]]
            print(self.attributes[key]['p'])
        print(self.attributes['IS']['p'])

        """
        for attr_values in self.attributes.values():
            attr_values['p'] = [len(item) / len(self.nodes)
                                for item in attr_values["cluster"]]
        """


In [5]:
f = Foo()
print(f.attributes)

f.set_attribute_probability()


IS
[0.008064516129032258, 0.008064516129032258, 0.008064516129032258, 0.008064516129032258, 0.03225806451612903, 0.9354838709677419]
OS
[0.008064516129032258, 0.016129032258064516, 0.008064516129032258, 0.056451612903225805, 0.07258064516129033, 0.8387096774193549]
BC
[0.008064516129032258, 0.008064516129032258, 0.008064516129032258, 0.008064516129032258, 0.016129032258064516, 0.9516129032258065]
DC
[0.016129032258064516, 0.016129032258064516, 0.04032258064516129, 0.14516129032258066, 0.10483870967741936, 0.6774193548387096]
CC
[0.008064516129032258, 0.016129032258064516, 0.12096774193548387, 0.1532258064516129, 0.5645161290322581, 0.13709677419354838]
E
[0.008064516129032258, 0.008064516129032258, 0.008064516129032258, 0.03225806451612903, 0.13709677419354838, 0.8064516129032258]
[0.008064516129032258, 0.008064516129032258, 0.008064516129032258, 0.008064516129032258, 0.03225806451612903, 0.9354838709677419]
